<img src="https://pages.cnpem.br/workshopbioimagens/wp-content/uploads/sites/166/2023/06/logo-ilum-2048x382.png" alt="Descrição da imagem" style="width: 1000px; height: auto; ">

<div style=" padding: 10px; font-size: 15px; text-align: center;">
<strong> 👹 Fera Formidável 4.9 👹 </strong> 
    </div>

<div style=" padding: 10px; font-size: 25px; text-align: center;">
<strong> A senha de tamanho variável </strong> 
    </div>
    
<div class="alert alert-warning">
  <div style="text-align: center; font-size: 15px"><b>Objetivo:</b>  Utilizar algoritmos genéticos para descobrir uma senha sem saber o seu tamanho para gerar a população do problema, apenas que tem entre 1 e 30 dígitos. </div>
</div>

<div style=" padding: 10px; font-size: 15px; text-align: center;">
<strong>Yasmin Barbosa Shimizu</strong></div>
<div style=" padding: 10px; font-size: 15px; text-align: center;">
Prof. Dr. Daniel R. Cassar</div>

## Introdução
> <i> Uma senha formada por letras maiúsculas, minúsculas ou dígitos numéricos deve ser encontrada. Cada palpite de senha retorna uma informação numérica que quantifica o quão perto o palpite está da senha correta. O critério de parada deste problema é quando a senha for descoberta. </i> --- Daniel Cassar, Notebook descobrindo a senha.

<div style=" padding: 10px; text-align: justify">
O problema acima foi descrito na quarta aula da disciplina de <i><b>Algoritmos Genéticos</b></i>, e tem o objetivo de descobrir uma senha com letras e números, tendo um tamanho fixo para a <i>string</i> que define a senha. Neste trabalho, aplicamos um problema semelhante a este, entretanto, sem dizer ao algoritmo o tamanho real da senha. Assim, encontramos três senhas de tamanho variável (uma longa, uma curta, e um caractere) de forma rápida e otimizada.
    </div>

## Metodologia

<div style=" padding: 10px; text-align: justify">
Entre os módulos <i>Python</i> utilizados, estão as bibliotecas <code>random</code> --- para sorteio das populações ---, <code>string</code> --- para definir os caracteres da senha ---, e <code>functools</code> --- visto que foi necessário definir uma função parcial. Além disso, criamos um <i>script</i> contendo funções utilizadas no problema da senha, da senha variável, bem como algoritmos de seleção, cruzamento e mutação, importando algumas delas para utilização neste notebook (<i>ver arquivo <code>funcoes_senha_tvar.py</code> no repositório GitHub</i>).
    </div>

In [1]:
import random
from string import ascii_lowercase, ascii_uppercase, digits
from functools import partial

from funcoes_senha_tvar import populacao_senha_tvar as cria_populacao
from funcoes_senha_tvar import funcao_objetivo_pop_senha_tvar
from funcoes_senha_tvar import selecao_torneio_min as funcao_selecao
from funcoes_senha_tvar import cruzamento_ponto_variavel as funcao_cruzamento
from funcoes_senha_tvar import mutacao_simples as funcao_mutacao1
from funcoes_senha_tvar import mutacao_salto as funcao_mutacao2
from funcoes_senha_tvar import mutacao_insercao_delecao as funcao_mutacao3

In [2]:
CARACTERES_POSSIVEIS = ascii_lowercase + ascii_uppercase + digits
funcao_objetivo = partial(funcao_objetivo_pop_senha_tvar, letras_possiveis=CARACTERES_POSSIVEIS)

In [3]:
TAMANHO_POPULACAO = 100
CHANCE_DE_CRUZAMENTO = 0.5
CHANCE_DE_MUTACAO = 0.025
TAMANHO_TORNEIO = 3

##### Modificações realizadas nas funções do problema da senha para o problema da senha variável
    
<ul style=" padding: 10px; text-align: justify">
<li> <b><code>populacao_senha_tvar</code></b>: criamos uma função <code>tamanho_senha_tvar</code> para sortear aleatoriamente o tamanho da senha seguindo o intervalo definido, a qual foi utilizada definir o tamanho da senha para gerar indivíduos na função <code>cria_candidato_senha_tvar</code> --- que por sua vez, é usada em loop para criar a população nessa função.

<li> <b><code>funcao_objetivo_pop_senha_tvar</code></b>: aqui, atribuímos uma "distância" para cada letra faltante ou sobrando no candidato, em relação à senha real. Como o tamanho da senha é extremamente importante para acertar a senha em si, atribuímos um valor de erro muito maior para a diferença de tamanho entre a senha verdadeira e o candidato em questão, de 1 + 10 vezes o espaço de busca para cada letra a mais ou a menos --- dessa forma, evitamos que o algoritmo encontre senhas muito próximas da verdadeira, apenas faltando ou sobrando um caractere, visto que a diferença de tamanho tem grande aumento na distância. Assim, foi necessário adicionar um argumento na função, correspondente aos caracteres possíveis para a senha, de modo que demandasse a contrução da função objetivo parcial, definindo como padrão letras minúsculas, maiúsculas e dígitos numéricos.

<li> <b><code>cruzamento_ponto_variavel</code></b>: essa função utiliza diferentes tipos de cruzamente a depender do tamano dos progenitores, e é baseada principalmente no cruzamento de ponto duplo. Como, para senhas muito pequenas, o cruzamento de ponto duplo pode não ser possível, definimos que ele só acontece se ambos os progenitores tiverem 5 ou mais genes. Quando pelo menos uma das senhas tem menos de 5 genes, ocorre um cruzamento de ponto simples, mas se pelo menos uma senha tiver apenas um gene, não há cruzamento. Além disso, os cortes foram definidos de acordo com o candidato de menos genes, de modo que um filho tem o mesmo número de genes da mãe, e outro o mesmo número de genes do pai - ou seja, não há variação de tamanho dos candidatos no cruzamento.

<li> <b><code>mutacao_insercao_delecao</code></b>: para poder variar o tamanho das senhas ao longo das gerações, foi implementado uma mutação de inserção ou deleção de genes. Caso a mutação ocorra, escolhe-se um índice na senha mutada, há um sorteio entre deleção e inserção e, respectivamente, retira-se o gene naquele índice, ou sorteia-se um gene aleatório dentre os caracteres possíveis para adicionar àquela posição, deslocando os demais genes.
</ul>    
 
<div style=" padding: 10px; text-align: justify"> 
Tentou-se ainda, implementar outras funções que não se mostraram tão efetivas, como realizar um cruzamento de ponto duplo para diferentes tamanhos, em que eram definidos dois cortes para cada progenitor, cruzando as partes <i>mãe1-pai2-mãe3</i> para um filho e a parte <i>pai1-mae2-pai3</i> para outro. Entretanto, a convergência foi muito mais demorada, levando inúmeras gerações para localizar a senha correta, já que o tamanho das senhas das novas gerações eram completamente diferentes da geração anterior, perdendo a assertividade no tamanho da senha que se deseja encontrar.
    </div>

<div style=" padding: 10px; text-align: justify">
Tendo definido as alterações necessárias, resolvemos o problema da senha para três situações: uma senha longa (27 caracteres), uma curta (4 caracteres) e para apenas um caractere --- correspondentes às três situações definidas no cruzamento de ponto variável.
    </div>

### Senha longa



Primeiramente, definimos nossa senha:

In [4]:
SENHA_LONGA = list("SomenteDiaSistemaFeudal2006")

27


Definimos também, a nossa população. Aqui, é possível ver que cada candidato tem um tamanho diferente, como esperado.

In [5]:
populacao = cria_populacao(TAMANHO_POPULACAO, CARACTERES_POSSIVEIS)

print("PRIMEIRA GERAÇÃO")
for i in range(100):
    print(f"Candidato {i+1}: ", "".join(populacao[i]))

PRIMEIRA GERAÇÃO
Candidato 1:  d7chdXi9pj
Candidato 2:  UMThLCpNtTL0D7TvSJUe5GqAa
Candidato 3:  ttFXOa8jQf
Candidato 4:  mIWowm
Candidato 5:  jsEc31Pb
Candidato 6:  yhj3EJK7r
Candidato 7:  SWaR9TESLxz2CJRiCzCKLQmr
Candidato 8:  U6WEKxLxH2yFq93Dgkpqb9
Candidato 9:  ZD8e
Candidato 10:  B5tOMBBrn8NUf7d
Candidato 11:  mYzzOee1980ulQTBKC
Candidato 12:  RkI5Ta6b74lIvZd8xKnUwaQrQ767bj
Candidato 13:  0lsIVFA4Z5U9UKlMgKi3slK8zLvoc
Candidato 14:  6gklWFfXn8JEF8IeQ
Candidato 15:  NV
Candidato 16:  dLoa1mNXBiO1SDYZ4FZUwIIK
Candidato 17:  EDWS0R6l3uEwbE0eOl7g2FSVE
Candidato 18:  2
Candidato 19:  Ut9KSM8726mr8fLAtw
Candidato 20:  h1ty7bm4kIbi
Candidato 21:  7jPyj6mKOgdVc0wB50
Candidato 22:  1ltrMSPvR2Y0U6rD0EQvEo
Candidato 23:  55ORLtg7yBlbjmkaKbKcNVEH
Candidato 24:  3w3HZpFDwDBG6FR
Candidato 25:  AwzLE2
Candidato 26:  efhKiYlR6IzX1
Candidato 27:  JnEMIoR0cGKh
Candidato 28:  tjmjo9r2G0sVI
Candidato 29:  aH0r8Efbi5FoDfsPNFHAhYmY
Candidato 30:  260Gz9ySE3Z
Candidato 31:  bq6csClV7Zdr
Candidato 32:  F7

Por fim, rodamos o algoritmo genético:

In [6]:
menor_fitness_geral = float("inf")
geracao = 0

while menor_fitness_geral != 0:
    
    # Seleção
    fitness = funcao_objetivo(populacao, SENHA_LONGA)        
    selecionados = funcao_selecao(populacao, fitness, TAMANHO_TORNEIO)
    
    # Cruzamento
    proxima_geracao = []
    for pai, mae in zip(selecionados[::2], selecionados[1::2]):
        individuo1, individuo2 = funcao_cruzamento(pai, mae, CHANCE_DE_CRUZAMENTO)
        proxima_geracao.append(individuo1)
        proxima_geracao.append(individuo2)
    
    # Mutação
    funcao_mutacao1(proxima_geracao, CHANCE_DE_MUTACAO, list(CARACTERES_POSSIVEIS))
    funcao_mutacao2(proxima_geracao, CHANCE_DE_MUTACAO, list(CARACTERES_POSSIVEIS))
    funcao_mutacao3(proxima_geracao, CHANCE_DE_MUTACAO, list(CARACTERES_POSSIVEIS))
    
    # Encerramento
    populacao = proxima_geracao
    geracao += 1
    
    fitness = funcao_objetivo(populacao, SENHA_LONGA)
    menor_fitness_observado = min(fitness)
    
    if menor_fitness_observado < menor_fitness_geral:
        menor_fitness_geral = menor_fitness_observado
        indice = fitness.index(menor_fitness_observado)
        candidato = populacao[indice]
        print(geracao, "".join(candidato))

1 aIZSbukonyFS7nOmTq2n0knGU6G
2 aIZSbukonyFhJxCng5XLNznGU6G
4 aIZSbukonXJJnnStDRrU0knGU6G
6 ilRVgukonySGobStDRrUaEtXHJ0
7 aIZSbukonXJJnnStDRrnajtXU6G
8 ilRhLkpNtTLbJxCng5rUaEtXH60
9 ilRSbukNqTSGonSAm5fmaEtXH60
10 ilRVfoVOdyFSunOAm5fnpknGU60
11 ZlRSbukNqTSGobStD5fmaEtXH60
12 ZlRSbukNqTSGonStD5fmaEtXH60
13 ilRSbukNqyFSunOng5fnpknGH60
14 ZlRSbukNqTLGobStg5fnpknGH60
15 ZlRSbukNqTSSubStg5rmpknGH60
16 ZlRTbukNqTSGunStg5fnpknGH60
17 ZlRSbukNqTSSunStm5flpknGH60
18 ZlRSbukNqTSGonOng5fnaknGH60
20 ZlRSbukNqTSSunStg5fnpknGH60
21 ZlRSbukNqTSSunOng5fnaknGH60
24 ZlRSbukNpTSSunSnc5fnaknGH50
27 ZlRSbukNqTSkunOng5fnaknGH60
28 ZlRSbukNqTSkunOnc5fnaknGH60
29 ZlRSbukNqTSkunSnc5fnaknGH60
30 ZlRSbukNqTSkunSnc5fnakmGH50
31 ZlrSbukNqTSkunSnc5fnaknGH50
32 ZlrSbukNqTSkunSnc5fnakmGH50
35 ZlrSbukNqTSkuuSnc5fnakmGH50
36 ZlrSbukMqTSkuuSnc5fnakmGH50
37 ZlrSbukNpTSkuuSnc5fnajmGH50
38 ZlrSbukNqZSkuuSnc5fnajmGH50
40 ZlrSbukNqZSkuuSnb5fnajmGH50
41 ZlrSbukNqZSkuuSnb5fnajmGG50
43 ZlrSbujNqZSkuuSnb5fnajmGG50
44 ZlrSdujNqZSk

### Senha curta

Primeiramente, definimos nossa senha:

In [7]:
SENHA_CURTA = list("Nn6a")

Definimos também, a nossa população. Aqui, é possível ver que cada candidato tem um tamanho diferente, como esperado.

In [8]:
populacao = cria_populacao(TAMANHO_POPULACAO, CARACTERES_POSSIVEIS)

print("PRIMEIRA GERAÇÃO")
for i in range(100):
    print(f"Candidato {i+1}: ", "".join(populacao[i]))

PRIMEIRA GERAÇÃO
Candidato 1:  MBo7MxcVxO
Candidato 2:  Nf
Candidato 3:  9x0P464qL3IZNHUHFxw1yn7tEzd
Candidato 4:  5GatUSEzy8
Candidato 5:  DXfqvZ4Dgy
Candidato 6:  khBvtQDeUEeeGFbmvZW3CX
Candidato 7:  vXNjLiXeoXm
Candidato 8:  9plZcN
Candidato 9:  TvHuD3GGieJTdS0MpFjpoBi
Candidato 10:  j351Bcq6tg9
Candidato 11:  WWoiGXpNMBOlqWi7hX
Candidato 12:  zT4fSrUVALi8KPUy
Candidato 13:  5P2QSm7x0ZgpvB2
Candidato 14:  2lLM
Candidato 15:  l9
Candidato 16:  lGq6
Candidato 17:  dnso7hmcR
Candidato 18:  ptaAEIC2AbzCLDsWO1a3j6KwjM
Candidato 19:  lzCiShcjWF4ZR
Candidato 20:  anBt4mrGvWq3w
Candidato 21:  Ovq5Dj5HqtT7
Candidato 22:  IUJpvLEqBvKdXm82qeHkOFVUuI
Candidato 23:  Yxtdwr9vqR
Candidato 24:  LYU
Candidato 25:  HreaXGyW
Candidato 26:  EbVqTNR7rgj67Hlp
Candidato 27:  42
Candidato 28:  a9
Candidato 29:  fCnj87NJ
Candidato 30:  YEIiGxK
Candidato 31:  3INIx
Candidato 32:  i
Candidato 33:  rUctNxHeObGBG2fMV4OTIZUec
Candidato 34:  wmzGrd8gUgeh96mXeT3U9K6j3
Candidato 35:  r
Candidato 36:  JYQ5xIVFkOVBfN

Por fim, rodamos o algoritmo genético:

In [9]:
menor_fitness_geral = float("inf")
geracao = 0

while menor_fitness_geral != 0:
    
    # Seleção
    fitness = funcao_objetivPor fim, rodamos o algoritmo genético:o(populacao, SENHA_CURTA)        
    selecionados = funcao_selecao(populacao, fitness, TAMANHO_TORNEIO)
    
    # Cruzamento
    proxima_geracao = []
    for pai, mae in zip(selecionados[::2], selecionados[1::2]):
        individuo1, individuo2 = funcao_cruzamento(pai, mae, CHANCE_DE_CRUZAMENTO)
        proxima_geracao.append(individuo1)
        proxima_geracao.append(individuo2)
    
    # Mutação
    funcao_mutacao1(proxima_geracao, CHANCE_DE_MUTACAO, list(CARACTERES_POSSIVEIS))
    funcao_mutacao2(proxima_geracao, CHANCE_DE_MUTACAO, list(CARACTERES_POSSIVEIS))
    funcao_mutacao3(proxima_geracao, CHANCE_DE_MUTACAO, list(CARACTERES_POSSIVEIS))
    
    # Encerramento
    populacao = proxima_geracao
    geracao += 1
    
    fitness = funcao_objetivo(populacao, SENHA_CURTA)
    menor_fitness_observado = min(fitness)
    
    if menor_fitness_observado < menor_fitness_geral:
        menor_fitness_geral = menor_fitness_observado
        indice = fitness.index(menor_fitness_observado)
        candidato = populacao[indice]
        print(geracao, "".join(candidato))

1 LlLM
3 Dl8L
4 Dl8M
5 Nl8L
7 Nl8M
9 Nl8R
12 Nl6R
13 Nl6f
17 Nm6e
20 Nn6e
24 Nn6d
29 Nn6c
33 Nn6b
42 Nn6a


### Senha de um caractere

Primeiramente, definimos nossa senha:

In [10]:
SENHA_Y = list("Y")

Definimos também, a nossa população. Aqui, é possível ver que cada candidato tem um tamanho diferente, como esperado.

In [11]:
populacao = cria_populacao(TAMANHO_POPULACAO, CARACTERES_POSSIVEIS)
Definimos também, a nossa população. Aqui, é possível ver que cada candidato tem um tamanho diferente, como esperado.
print("PRIMEIRA GERAÇÃO")
for i in range(100):
    print(f"Candidato {i+1}: ", "".join(populacao[i]))

PRIMEIRA GERAÇÃO
Candidato 1:  WtH4cYrQtqiFC02
Candidato 2:  qCxSbtn
Candidato 3:  0wyISMcHbLxIwtGzIbup06
Candidato 4:  UfzThG
Candidato 5:  dkAGfKUnxNkA6psOJCi6fP9bmKBn
Candidato 6:  ooW9MIIqyaeu
Candidato 7:  CzEWspPUJ04jKmZL8yikAbEie5Y70
Candidato 8:  2uSE4ksy4Lv
Candidato 9:  4ulWTaExwwDBDTX0m70SH5WZD1N4h
Candidato 10:  4QL
Candidato 11:  ThFchc3bceIa7Cj20g
Candidato 12:  xbUExRdbFklOyN
Candidato 13:  yIWlw5sCFpXArrnQFs
Candidato 14:  S2lrYybogTDGj7rMk9jRX2
Candidato 15:  2gNFsMBoqBBbSD33YAADT5P9Gl2hGg
Candidato 16:  sEmRsrJ
Candidato 17:  uoFUCoTtEPw1IsQRCgYWHofjn
Candidato 18:  4ldGbLJKpNC2IuYXg
Candidato 19:  qZtUYjXULCD4f48VLeaqiidCerScbK
Candidato 20:  4U9Utewo6sshCZjVZ2QH
Candidato 21:  gQj
Candidato 22:  D
Candidato 23:  KwtS
Candidato 24:  7b68EGkS33dIxrZjcjxC9utdApjyx
Candidato 25:  NIWtmVMVq91X
Candidato 26:  2Ob1ljjORvCQw0lT5kh8otMw01WQz
Candidato 27:  XVvfXwTySUHYgjosEf6haYlki
Candidato 28:  imnabJQVSrWZGJE2WcJ64IDxS
Candidato 29:  1uROvBL3A
Candidato 30:  I
Candidato 3

Por fim, rodamos o algoritmo genético:

In [12]:
menor_fitness_geral = float("inf")
geracao = 0

while menor_fitness_geral != 0:
    
    # Seleção
    fitness = funcao_objetivo(populacao, SENHA_Y)        
    selecionados = funcao_selecao(populacao, fitness, TAMANHO_TORNEIO)
    
    # Cruzamento
    proxima_geracao = []
    for pai, mae in zip(selecionados[::2], selecionados[1::2]):
        individuo1, individuo2 = funcao_cruzamento(pai, mae, CHANCE_DE_CRUZAMENTO)
        proxima_geracao.append(individuo1)
        proxima_geracao.append(individuo2)
    
    # Mutação
    funcao_mutacao1(proxima_geracao, CHANCE_DE_MUTACAO, list(CARACTERES_POSSIVEIS))
    funcao_mutacao2(proxima_geracao, CHANCE_DE_MUTACAO, list(CARACTERES_POSSIVEIS))
    funcao_mutacao3(proxima_geracao, CHANCE_DE_MUTACAO, list(CARACTERES_POSSIVEIS))
    
    # Encerramento
    populacao = proxima_geracao
    geracao += 1
    
    fitness = funcao_objetivo(populacao, SENHA_YPor fim, rodamos o algoritmo genético:)
    menor_fitness_observado = min(fitness)
    
    if menor_fitness_observado < menor_fitness_geral:
        menor_fitness_geral = menor_fitness_observado
        indice = fitness.index(menor_fitness_observado)
        candidato = populacao[indice]
        print(geracao, "".join(candidato))

1 V
8 Z
63 Y


## Conclusão

<div style=" padding: 10px; text-align: justify">
Neste trabalho, foi desenvolvido um algoritmo genético que descobre uma <i>string</i> de até 30 caracteres. Para isso, nos baseamos no problema da senha (com tamanho conhecido) solucionado nas aulas da disciplina de Algoritmos Genéticos. Assim, implementamos novas funções --- <code>populacao_senha_tvar</code>, <code>funcao_objetivo_pop_senha_tvar</code>, <code>cruzamento_ponto_variavel</code> e <code>mutacao_insercao_delecao</code> --- e tentamos descobrir uma senha longa, uma senha curta, e apenas um caractere. Desse modo, o algoritmo se mostrou bastante eficiente e otimizado para todos os casos em comparação a testes realizados ao longo de sua construção --- antes da mudança da função de cruzamento e implementação mutação de deleção ou inserção, senhas pequenas eram quase impossíveis de serem encontradas ---, descobrindo as senhas verdadeiras em poucas gerações.
    
</div>

### Referências

Aulas ministradas pelo Prof. Dr. Daniel R. Cassar na disciplina ATP-303 Redes Neurais e Algoritmos Genéticos, na Ilum Escola de Ciência.